In [13]:
import sys
sys.path.append('../nimble_challenge')
sys.path.append('./ctpn/src/')
# sys.path.append('/home/arka_s/Caffe/cf/caffe/python/')
# import caffe
import two_imgs_eff
import pickle
from txt_data_storer import info_storer, info_storer_all
# import detectors
# import other

In [14]:
import skimage.io as skio
import skimage.draw as skd
%matplotlib
import os
import numpy as np

Using matplotlib backend: TkAgg


In [66]:
tdir = '/home/nkovvuri/Rama_Work/dataset/Protest_Images/'
spdir = 'Modified_Images_ProtestL/'
swdir = 'Pruned_Protest_YFCCImages/'
npy_file = 'mod_fine_labels.npy'
pimg_list = os.listdir(tdir + spdir)
pimg_list = [i for i in pimg_list if i[-4:] == '.jpg' or i[-4:]=='.png']
test_img = pimg_list[0]
p_img = skio.imread(tdir + spdir + test_img)
actual_bbx = np.load(tdir + npy_file).item()[test_img[:-4]]
actual_bbx[0], actual_bbx[1] = actual_bbx[1], actual_bbx[0]
actual_bbx[2], actual_bbx[3] = actual_bbx[3], actual_bbx[2]
w_img = skio.imread(tdir + swdir + 'protest_img_' + test_img.split('_')[-1])
abx = np.array((
    (actual_bbx[0], actual_bbx[1]),
    (actual_bbx[0], actual_bbx[3]),
    (actual_bbx[2], actual_bbx[3]),
    (actual_bbx[2], actual_bbx[1]),
))
rr, cc = skd.polygon_perimeter(abx[:,0], abx[:,1], p_img.shape, clip=True)
p_img_bbx = get_img_with_bbx(p_img, rr, cc, copy=True)
p_img_only_bbx = p_img[actual_bbx[0]:actual_bbx[2], actual_bbx[1]:actual_bbx[3]]
skio.imshow_collection([p_img_bbx, p_img_only_bbx, w_img])
# skio.imshow_collection([p_img_only_bbx, w_img])
# p_img_only_bbx.shape
# rr.shape

In [31]:
def get_cropped(img, bbox,default=True):
    '''
    Assuming format is (x1, y1, x2, y2)
    '''
    bbx = bbox.astype(int)
    if default:
        img1 = img[bbx[0]:bbx[2], bbx[1]:bbx[3], :]
    else:
        img1 = img[bbx[1]:bbx[3], bbx[0]:bbx[2], :]
    return img1

In [63]:
def get_img_with_bbx(img, rr, cc, copy=False, color=(255, 0, 0)):
    if copy:
        img1 = img.copy()
    else:
        img1 = img
    skd.set_color(img1, (rr, cc), color)
    return img1

In [67]:
def get_img_with_bbx2(img, actual_bbx2, default=False, copy=False, color=(255, 0, 0)):
    actual_bbx = actual_bbx2.copy()
    if default:
        actual_bbx[0], actual_bbx[1] = actual_bbx[1], actual_bbx[0]
        actual_bbx[2], actual_bbx[3] = actual_bbx[3], actual_bbx[2]
    abx = np.array((
    (actual_bbx[0], actual_bbx[1]),
    (actual_bbx[0], actual_bbx[3]),
    (actual_bbx[2], actual_bbx[3]),
    (actual_bbx[2], actual_bbx[1]),
    ))
    rr, cc = skd.polygon_perimeter(abx[:,0], abx[:,1], p_img.shape, clip=True)
    if copy:
        img1 = img.copy()
    else:
        img1 = img
    skd.set_color(img1, (rr, cc), color)
    return img1

In [70]:
# Text Boxes
dict_bbx = pickle.load(open('../../data/protest_data/dict_Modified_Images_ProtestL_text_lines.pkl'))
dict_bbx[test_img]
# skio.imshow(get_cropped(p_img, dict_bbx[test_img][1],default=False))
skio.imshow(get_img_with_bbx2(p_img, dict_bbx[test_img][0], default=True, copy=True))
# bx = dict_bbx[test_img][1].astype(int)
# print(bx)
# p1 = p_img[bx[0]:bx[2], bx[1]:bx[3]]
# p1 = p_img[bx[1]:bx[3], bx[0]:bx[2]]
# skio.imshow(p1)
# dict_bbx[test_img][0].astype(int)

In [96]:

# print(actual_bbx)
# p_img.shape
# poly = np.array((
#     (300, 300),
#     (480, 320),
#     (380, 430),
#     (220, 590),
#     (300, 300),
# ))
# rr, cc = polygon_perimeter(poly[:, 0], poly[:, 1], p_img.shape)

# p_img_bbx = p_img.copy()
# p_img_bbx[rr, cc, 0] = 255
# p_img_bbx[rr, cc, 1] = 0
# p_img_bbx[rr, cc, 2] = 0



[ 621  657  944 1227]


In [92]:
help(skd.set_color)

Help on function set_color in module skimage.draw.draw:

set_color(img, coords, color, alpha=1)
    Set pixel color in the image at the given coordinates.
    
    Coordinates that exceed the shape of the image will be ignored.
    
    Parameters
    ----------
    img : (M, N, D) ndarray
        Image
    coords : tuple of ((P,) ndarray, (P,) ndarray)
        Row and column coordinates of pixels to be colored.
    color : (D,) ndarray
        Color to be assigned to coordinates in the image.
    alpha : scalar or (N,) ndarray
        Alpha values used to blend color with image.  0 is transparent,
        1 is opaque.
    
    Returns
    -------
    img : (M, N, D) ndarray
        The updated image.
    
    Examples
    --------
    >>> from skimage.draw import line, set_color
    >>> img = np.zeros((10, 10), dtype=np.uint8)
    >>> rr, cc = line(1, 1, 20, 20)
    >>> set_color(img, (rr, cc), 1)
    >>> img
    array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
           [0, 1, 0, 0, 0, 0, 0, 

In [91]:
print(abx)
poly

[[ 621  657]
 [ 621 1227]
 [ 944 1227]
 [ 944  657]]


array([[300, 300],
       [480, 320],
       [380, 430],
       [220, 590],
       [300, 300]])

In [28]:
# actual_bbx
# test_img.split('_')[-1]
actual_bbx

array([ 657,  621, 1227,  944], dtype=int32)

In [21]:
from jupyter_core.paths import jupyter_config_dir
jupyter_dir = jupyter_config_dir()
jupyter_dir

'/home/arka_s/.jupyter'

In [18]:
import math
import numpy as np
import matplotlib.pyplot as plt

from skimage.draw import (line, polygon, circle,
                          circle_perimeter,
                          ellipse, ellipse_perimeter,
                          bezier_curve, polygon_perimeter)


fig, ax1 = plt.subplots(ncols=1, nrows=1, figsize=(10, 6))


img = np.zeros((500, 500, 3), dtype=np.double)

# fill polygon
poly = np.array((
    (300, 300),
    (480, 320),
    (380, 430),
    (220, 590),
    (300, 300),
))
rr, cc = polygon_perimeter(poly[:, 0], poly[:, 1], img.shape)
img[rr, cc, 1] = 1


ax1.imshow(img)
ax1.set_title('No anti-aliasing')
ax1.axis('off')


(-0.5, 499.5, 499.5, -0.5)